In [ ]:
%load_ext autotime
# %unload_ext autotime

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import soundfile as sf
import torch
from jiwer import wer

from sonorus.speech.lm import (
    FairseqTokenDictionary,
    W2lKenLMDecoder,
    W2lViterbiDecoder,
    W2lFairseqLMDecoder,
)

In [ ]:
librispeech_eval = load_dataset("librispeech_asr", "clean", 
                                split="test", 
                                ignore_verifications=True)#,
                                # download_mode="force_redownload")

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

In [ ]:
librispeech_eval = librispeech_eval.map(map_to_array)

In [ ]:
def map_to_pred(batch):
    input_values = tokenizer(batch["speech"], return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

In [ ]:
result = librispeech_eval.map(map_to_pred, batched=True, batch_size=1, remove_columns=["speech"])
# print("WER:", wer(result["text"], result["transcription"]))

In [ ]:
# import pickle
# with open("hf_transcriptions.pk", "wb") as f:
#     pickle.dump(result, f)
# with open("hf_transcriptions.pk", "rb") as f:
#     result = pickle.load(f)
# print("WER:", wer(result["text"][1000:], result["transcription"][1000:]))

In [ ]:
def map_to_pred_lm(batch):
    input_values = tokenizer(batch["speech"], return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits

    logits = logits.float().cpu().contiguous()
    decoded = decoder.decode(logits)
    # 1st sample, 1st best transcription
    transcription = decoder.post_process(decoded)
    batch["transcription"] = transcription
    return batch

In [ ]:
token_dict = FairseqTokenDictionary(
    indexed_symbols=tokenizer.get_vocab()
)

In [ ]:
lexicon_path = "/home/harold/Documents/IISc-work/imperio/data/speech/fairseq/librispeech_lexicon.lst"
lm_path = "/home/harold/Documents/IISc-work/imperio/data/speech/fairseq/lm_librispeech_kenlm_word_4g_200kvocab.bin"

decoder = W2lKenLMDecoder(
    token_dict=token_dict,
    lexicon=lexicon_path,
    lang_model=lm_path,
    beam=1500,
    beam_size_token=100,
    beam_threshold=25,
    lm_weight=1.5,
    word_weight=-1,
    unk_weight=float("-inf"),
    sil_weight=0,
)

In [ ]:
result = librispeech_eval.map(map_to_pred_lm, batched=True, batch_size=1, remove_columns=["speech"])
# print("WER:", wer(result["text"], 
#                   [x[0] for x in result["transcription"]]))

In [ ]:
print("WER:", wer(result["text"][1000:], 
                  [x[0] for x in result["transcription"]][1000:]))